In [89]:
import lkh
from timeit import default_timer as timer
from CVRP import (
    Trucks, parse_vrp_file, calculate_distance_matrix, 
    create_initial_solution, total_route_cost, tabu_search
)
from amplpy import AMPL, add_to_path

add_to_path(r"C:\Path\To\AMPL\CommunityEdition")

In [110]:
lista_instancias = ['Instances\\P-n16-k8.vrp','Instances\\P-n19-k2.vrp','Instances\\P-n20-k2.vrp','Instances\\P-n21-k2.vrp','Instances\\P-n22-k2.vrp']
tiempos_lkh = []
resultados_lkh = []
soluciones_lkh = []
tiempos_tabu = []
resultados_tabu = []
matrices_de_distancia = []

In [111]:
for instancia in lista_instancias:
  file_path = instancia
  capacity, clients = parse_vrp_file(file_path)
  sumDemand = sum(client.demand for client in clients)
  numTrucks = sumDemand // capacity
  trucks = []
  for i in range(numTrucks + 3):
      trucks.append(Trucks(id=i + 1, capacity=capacity))
  dist_matrix = calculate_distance_matrix(clients)
  matrices_de_distancia.append(dist_matrix)
  create_initial_solution(trucks, clients, dist_matrix)
  initial_solution = []
  for truck in trucks:
      initial_solution.append(truck.route)
  initial_cost = total_route_cost(initial_solution, dist_matrix)
  start = timer()
  opt_solution, opt_cost = tabu_search(initial_solution, dist_matrix, 100, capacity)
  end = timer()
  tiempos_tabu.append(end-start)
  resultados_tabu.append(opt_cost)

Can't add more clients Truck(id=1, capacity=35, packages=31, route=[1, 7])
Can't add more clients Truck(id=2, capacity=35, packages=33, route=[1, 2, 11, 14])
Can't add more clients Truck(id=3, capacity=35, packages=30, route=[1, 3])
Can't add more clients Truck(id=4, capacity=35, packages=30, route=[1, 5, 12])
Can't add more clients Truck(id=5, capacity=35, packages=34, route=[1, 8, 6, 10])
Can't add more clients Truck(id=6, capacity=35, packages=25, route=[1, 16, 13])
Can't add more clients Truck(id=7, capacity=35, packages=35, route=[1, 15, 4])
New best solution ever [[1, 7, 1], [1, 2, 11, 1], [1, 3, 1], [1, 5, 12, 1], [1, 8, 6, 10, 1], [1, 16, 13, 1], [1, 15, 4, 1], [1, 14, 9, 1], [1, 1], [1, 1]], 479.4918143290543
0.Best solution so far [[1, 7, 1], [1, 2, 11, 1], [1, 3, 1], [1, 5, 12, 1], [1, 8, 6, 10, 1], [1, 16, 13, 1], [1, 15, 4, 1], [1, 14, 9, 1], [1, 1], [1, 1]], 479.4918143290543
New best solution ever [[1, 7, 1], [1, 2, 1], [1, 3, 1], [1, 5, 12, 1], [1, 8, 6, 10, 1], [1, 16,

In [112]:
print(resultados_tabu)
print(tiempos_tabu)

[451.9470921298186, 272.6685818082292, 254.12718045669294, 239.00798285439762, 254.9611268706774]
[0.07140790001722053, 0.04832999996142462, 0.05382400000235066, 0.1166240000165999, 0.03444409999065101]


In [113]:
for instancia in lista_instancias:
  file_path = instancia
  with open(file_path, 'r') as file:
      problem_str = file.read()
  problem = lkh.LKHProblem.parse(problem_str)


  solver_path = 'LKH-3.exe'

  start = timer()
  a = lkh.solve(solver_path, problem=problem, max_trials=10000, runs=1)
  end = timer()
  tiempos_lkh.append(end - start)
  soluciones_lkh.append(a)

In [114]:
print(tiempos_lkh)
print(soluciones_lkh)

[3.0447529000230134, 1.6174961000215262, 1.7507563999970444, 1.404033900005743, 1.3007110999897122]
[[[2, 4], [3], [11, 13, 16], [15, 6], [8, 10, 14], [9], [5, 12], [7]], [[7, 9, 17, 18, 4, 13, 15, 12, 5], [2, 11, 3, 8, 10, 16, 14, 6, 19]], [[2, 11, 3, 8, 10, 17, 15, 6, 20], [5, 12, 16, 13, 4, 19, 18, 9, 14, 7]], [[7, 8, 3, 14, 10, 18, 15, 6, 21], [5, 12, 16, 13, 4, 20, 19, 9, 11, 2, 17]], [[7, 8, 3, 14, 10, 22, 18, 15, 6, 21], [5, 12, 16, 13, 4, 20, 19, 9, 11, 2, 17]]]


In [115]:

def solution_cost(solution,dist_matrix):
    traveled_distance = 0
    for route in solution:
        for i in range(len(route) - 1):
            traveled_distance += dist_matrix[route[i]-1][route[i + 1]-1]
    return traveled_distance

In [116]:
for solution in soluciones_lkh:
  for route in solution:
    route.insert(0,1)
    route.append(1)
  print(solution)

[[1, 2, 4, 1], [1, 3, 1], [1, 11, 13, 16, 1], [1, 15, 6, 1], [1, 8, 10, 14, 1], [1, 9, 1], [1, 5, 12, 1], [1, 7, 1]]
[[1, 7, 9, 17, 18, 4, 13, 15, 12, 5, 1], [1, 2, 11, 3, 8, 10, 16, 14, 6, 19, 1]]
[[1, 2, 11, 3, 8, 10, 17, 15, 6, 20, 1], [1, 5, 12, 16, 13, 4, 19, 18, 9, 14, 7, 1]]
[[1, 7, 8, 3, 14, 10, 18, 15, 6, 21, 1], [1, 5, 12, 16, 13, 4, 20, 19, 9, 11, 2, 17, 1]]
[[1, 7, 8, 3, 14, 10, 22, 18, 15, 6, 21, 1], [1, 5, 12, 16, 13, 4, 20, 19, 9, 11, 2, 17, 1]]


In [117]:
print(soluciones_lkh)
i = 0
for solution in soluciones_lkh:
  resultados_lkh.append(solution_cost(solution, matrices_de_distancia[i]))
  i+=1

[[[1, 2, 4, 1], [1, 3, 1], [1, 11, 13, 16, 1], [1, 15, 6, 1], [1, 8, 10, 14, 1], [1, 9, 1], [1, 5, 12, 1], [1, 7, 1]], [[1, 7, 9, 17, 18, 4, 13, 15, 12, 5, 1], [1, 2, 11, 3, 8, 10, 16, 14, 6, 19, 1]], [[1, 2, 11, 3, 8, 10, 17, 15, 6, 20, 1], [1, 5, 12, 16, 13, 4, 19, 18, 9, 14, 7, 1]], [[1, 7, 8, 3, 14, 10, 18, 15, 6, 21, 1], [1, 5, 12, 16, 13, 4, 20, 19, 9, 11, 2, 17, 1]], [[1, 7, 8, 3, 14, 10, 22, 18, 15, 6, 21, 1], [1, 5, 12, 16, 13, 4, 20, 19, 9, 11, 2, 17, 1]]]


In [119]:

print(resultados_lkh)

[451.9470921298186, 212.65690418589227, 217.41558334616656, 212.7115397029039, 217.869528478106]


In [120]:
lista_instancias_ampl = ['Instances\\P-n16-k8.dat','Instances\\P-n19-k2.dat','Instances\\P-n20-k2.dat','Instances\\P-n21-k2.dat','Instances\\P-n22-k2.dat']
tiempos_ampl = []
resultados_ampl = []

In [121]:
for instancia in lista_instancias_ampl:
  ampl = AMPL()
  # ampl.reset()
  ampl.read('CVRP.mod')

  # Load the data file
  ampl.readData(instancia)

  # Set the solver to CPLEX or Gurobi
  ampl.setOption('solver', 'gurobi')  # for Gurobi
  start = timer()
  ampl.solve()
  end = timer()
  tiempos_ampl.append(end - start)
  z = ampl.getObjective('z').value()
  resultados_ampl.append(z)
  x = ampl.getVariable('x').getValues()
  u = ampl.getVariable('u').getValues()

Error:
	Error executing "solve" command:
	error processing param demand:
		invalid subscript demand[1] discarded.

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>

Gurobi 11.0.2:Gurobi 11.0.2: optimal solution; objective 451.3350802
93558 simplex iterations
5161 branching nodes
Error:
	Error executing "solve" command:
	error processing param demand:
		invalid subscript demand[1] discarded.

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>

Gurobi 11.0.2:Gurobi 11.0.2: optimal solution; objective 212.6569042
772378 simplex iterations
64529 branching nodes
Error:
	Error executing "solve" command:
	error processing param demand:
		invalid subscript demand[1] discarded.

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>

Gurobi 11.0.2:Gurobi 11.0.2: optimal solution; objective 217.4155833
246526 simplex iterations
19656 branching nodes
Error:
	Error executing "solve" command:
	error processing p

In [122]:
print(lista_instancias)
print(tiempos_ampl)
print(resultados_ampl)

['P-n16-k8.vrp', 'P-n19-k2.vrp', 'P-n20-k2.vrp', 'P-n21-k2.vrp', 'P-n22-k2.vrp']
[3.0168967999634333, 13.199951799993869, 5.968637199955992, 2.9264645999646746, 8.190742799954023]
[451.335080208618, 212.6569041858923, 217.41558334616658, 212.71153970290385, 217.85215074022966]


In [125]:
print("Tiempos-tabu",tiempos_tabu)
print("Tiempos-lkh",tiempos_lkh)
print("Tiempos AMPL-gurobi",tiempos_ampl)
print("Resultados-tabu",resultados_tabu)
print("Resultados-lkh",resultados_lkh)
print("Resultados-ampl",resultados_ampl)

Tiempos-tabu [0.07140790001722053, 0.04832999996142462, 0.05382400000235066, 0.1166240000165999, 0.03444409999065101]
Tiempos-lkh [3.0447529000230134, 1.6174961000215262, 1.7507563999970444, 1.404033900005743, 1.3007110999897122]
Tiempos AMPL-gurobi [3.0168967999634333, 13.199951799993869, 5.968637199955992, 2.9264645999646746, 8.190742799954023]
Resultados-tabu [451.9470921298186, 272.6685818082292, 254.12718045669294, 239.00798285439762, 254.9611268706774]
Resultados-lkh [451.9470921298186, 212.65690418589227, 217.41558334616656, 212.7115397029039, 217.869528478106]
Resultados-ampl [451.335080208618, 212.6569041858923, 217.41558334616658, 212.71153970290385, 217.85215074022966]


In [127]:
import pandas as pd


In [142]:
nombre_instancias = ['P-n16-k8','P-n19-k2','P-n20-k2','P-n21-k2','P-n22-k2']
data = {
    'Instancias' : nombre_instancias,
    'Tiempos-tabu': tiempos_tabu,
    'Tiempos-lkh': tiempos_lkh,
    'Tiempos AMPL-gurobi': tiempos_ampl,
    'Resultados-tabu': resultados_tabu,
    'Resultados-lkh': resultados_lkh,
    'Resultados-ampl': resultados_ampl
}

df = pd.DataFrame(data)
print(df)
df.to_csv('results_table.csv', index=False)

     Instancias  Tiempos-tabu  Tiempos-lkh  Tiempos AMPL-gurobi  \
0  P-n16-k8.vrp      0.071408     3.044753             3.016897   
1  P-n19-k2.vrp      0.048330     1.617496            13.199952   
2  P-n20-k2.vrp      0.053824     1.750756             5.968637   
3  P-n21-k2.vrp      0.116624     1.404034             2.926465   
4  P-n22-k2.vrp      0.034444     1.300711             8.190743   

   Resultados-tabu  Resultados-lkh  Resultados-ampl  
0       451.947092      451.947092       451.335080  
1       272.668582      212.656904       212.656904  
2       254.127180      217.415583       217.415583  
3       239.007983      212.711540       212.711540  
4       254.961127      217.869528       217.852151  


In [144]:
df

,Instancias,Tiempos-tabu,Tiempos-lkh,Tiempos AMPL-gurobi,Resultados-tabu,Resultados-lkh,Resultados-ampl
0,P-n16-k8.vrp,0.071408,3.044753,3.016897,451.947092,451.947092,451.335080
1,P-n19-k2.vrp,0.048330,1.617496,13.199952,272.668582,212.656904,212.656904
2,P-n20-k2.vrp,0.053824,1.750756,5.968637,254.127180,217.415583,217.415583
3,P-n21-k2.vrp,0.116624,1.404034,2.926465,239.007983,212.711540,212.711540
4,P-n22-k2.vrp,0.034444,1.300711,8.190743,254.961127,217.869528,217.852151
